In [1]:
import pandas as pd
from datasets import Dataset
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments,DataCollatorForLanguageModeling
from datasets import load_from_disk
import torch
from torch.nn.utils.rnn import pad_sequence

c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('./Mental_Health_FAQ.csv')
df['input_output'] = df['Questions'] + df['Answers']
dataset = Dataset.from_pandas(df[['input_output']])

In [3]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
model = GPT2LMHeadModel.from_pretrained("gpt2")

In [4]:
def tokenize_function(data):
    return tokenizer(data['input_output'], padding="max_length", truncation=True, max_length=512)

tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])

Map: 100%|██████████| 98/98 [00:00<00:00, 384.09 examples/s]


In [5]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

In [6]:
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_steps=10_000,
    save_total_limit=2,
    logging_dir='./logs',
    logging_steps=200,
)

In [7]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
)

In [8]:
trainer.train()

100%|██████████| 147/147 [21:45<00:00,  8.88s/it]

{'train_runtime': 1305.445, 'train_samples_per_second': 0.225, 'train_steps_per_second': 0.113, 'train_loss': 2.58566055816858, 'epoch': 3.0}


TrainOutput(global_step=147, training_loss=2.58566055816858, metrics={'train_runtime': 1305.445, 'train_samples_per_second': 0.225, 'train_steps_per_second': 0.113, 'total_flos': 76819857408000.0, 'train_loss': 2.58566055816858, 'epoch': 3.0})

In [9]:
model.save_pretrained("./fine_tuned_gpt2")
tokenizer.save_pretrained("./fine_tuned_gpt2")

('./fine_tuned_gpt2\\tokenizer_config.json',
 './fine_tuned_gpt2\\special_tokens_map.json',
 './fine_tuned_gpt2\\vocab.json',
 './fine_tuned_gpt2\\merges.txt',
 './fine_tuned_gpt2\\added_tokens.json')